In [6]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/federico/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [7]:
alerts = await network_tools.get_alerts(time_window=2)
len(alerts.get('security_events', []).get('alerts'))

13

In [9]:
alerts = await network_tools.get_alerts(time_window=5)
alerts

{'security_events': {'alerts': [{'category': 'Potentially Bad Traffic',
    'dest_ip': '172.20.0.2',
    'dest_port': 3306,
    'flow_id': 1005756097835641,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'ET SCAN Suspicious inbound to mySQL port 3306',
    'src_ip': '192.168.100.2',
    'src_port': 53753,
    'timestamp': '2025-10-15T19:51:31.955066+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.2',
    'dest_port': 137,
    'flow_id': 1006920177658963,
    'payload': None,
    'proto': 'TCP',
    'severity': 3,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192.168.100.2',
    'src_port': 53753,
    'timestamp': '2025-10-15T19:51:31.955337+0200'},
   {'category': 'Potentially Bad Traffic',
    'dest_ip': '172.20.0.2',
    'dest_port': 1433,
    'flow_id': 1006990850365110,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'ET SCAN Suspicious inbound to MSSQL port 1433',
    'src_ip

In [4]:
network_tools.get_docker_containers()

{'honeypot_config': [{'service': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip': '172.20.0.2'}]}

In [5]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [31]:
await firewall_tools.remove_firewall_rule([1, 2])

INFO:tools.firewall_tools:Removing firewall rules: [1, 2]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  172.20.0.2           192.168.100.0/24',
  'REMOVED rule #2:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.2']}

In [93]:
honeypot_ip = '172.20.0.2' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip, port=8080)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.2 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)'}